In [1]:
import pandas as pd
import numpy as np
import requests
import os
from sklearn.cluster import KMeans
import folium 
import matplotlib.cm as cm
import matplotlib.colors as color
import scipy as sp

In [2]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0

The following packages will be UPDATED:

  openssl                                 1.1.1f-h516909a_0 --> 1.1.1g-h516909a_0



openssl-1.1.1g       | 2.1 MB    | ############

In [3]:
df_pitt = pd.read_csv('/resources/data/Pittsburgh.csv')
print('Dataset downloaded and read into a pandas dataframe!')

Dataset downloaded and read into a pandas dataframe!


In [4]:
df_pitt.head()

,PK,CCR,HIERARCHY,INCIDENTTIME,INCIDENTLOCATION,CLEAREDFLAG,INCIDENTNEIGHBORHOOD,INCIDENTZONE,INCIDENTHIERARCHYDESC,OFFENSES,INCIDENTTRACT,COUNCIL_DISTRICT,PUBLIC_WORKS_DIVISION,X,Y
0,2911979,17082758,3,2017-05-06T23:17:00,"Allegheny SQ Pittsburgh, PA 15212",N,Allegheny Center,1,ROBBERY/PURSE SNATCH/STRONGARM,3701 Robbery.,2204,1.0,6.0,-80.018015,40.448954
1,2875345,16221967,3,2016-11-26T19:17:00,"Allegheny SQ Pittsburgh, PA 15212",N,Allegheny Center,1,ROBBERY/HIGHWAY/STRONGARM,3701 Robbery.,2204,1.0,6.0,-80.018015,40.448954
2,2856393,16165138,5,2016-09-02T02:06:00,"Allegheny SQ Pittsburgh, PA 15212",Y,Allegheny Center,1,BURGLARY/FORCE ENT/NON-RES,3304 Criminal Mischief. / 3502 Burglary.,2204,1.0,6.0,-80.018015,40.448954
3,3014202,18141605,3,2018-07-24T23:00:00,"400 Block West Commons Pittsburgh, PA 15212",N,Allegheny Center,1,ROBBERY/HIGHWAY/STRONGARM,3701 Robbery.,2204,1.0,1.0,-80.005781,40.450162
4,3025101,18175232,3,2018-09-07T19:09:00,"Federal ST & South Commons Pittsburgh, PA 15212",N,Allegheny Center,1,ROBBERY/HIGHWAY/STRONGARM,903 Criminal Conspiracy. / 2701 Simple Assault...,2204,1.0,1.0,-80.005390,40.450229


In [5]:
pitt_crime_rank=df_pittcrime = df_pitt.groupby(['INCIDENTNEIGHBORHOOD']).count().reset_index()

In [6]:
pitt_crime_rank.drop(['CCR','HIERARCHY','INCIDENTTIME','INCIDENTLOCATION', 'CLEAREDFLAG','INCIDENTZONE',
                   'INCIDENTHIERARCHYDESC','OFFENSES','INCIDENTTRACT','COUNCIL_DISTRICT', 'PUBLIC_WORKS_DIVISION',
                  'OFFENSES','X','Y'], axis=1, inplace=True)


In [7]:
pitt_crime_rank.rename(columns={'INCIDENTNEIGHBORHOOD':'Neighborhood', 'PK':'Count'}, inplace=True)

In [8]:
pitt_crime_rank.sort_values(["Count"], axis=0, ascending=[False], inplace=True) 
pitt_crime_rank

,Neighborhood,Count
78,South Side Flats,551
15,Carrick,545
16,Central Business District,435
45,Knoxville,381
43,Homewood South,380
...,...,...
73,Ridgemont,5
57,Mt. Oliver Neighborhood,4
65,Outside State,3
58,New Homestead,3


In [8]:
pitt_crime_rank.sort_values(["Count"], axis=0, ascending=[False], inplace=True) 
pitt_crime_rank

,Neighborhood,Count
78,South Side Flats,551
15,Carrick,545
16,Central Business District,435
45,Knoxville,381
43,Homewood South,380
...,...,...
73,Ridgemont,5
57,Mt. Oliver Neighborhood,4
65,Outside State,3
58,New Homestead,3


In [9]:
pitt_crime_rank.describe()

,Count
count,98.000000
mean,128.857143
std,123.010183
min,1.000000
25%,30.000000
50%,98.000000
75%,201.000000
max,551.000000


In [10]:
pitt_police = pd.read_csv('/resources/data/Police Stations in Pittsburgh.csv')

In [11]:
pitt_police.drop(['id','parcel_id','rentable','type', 'primary_user','address_number',
                   'image','zip','pli_division','public_works_division', 'tract',
                  'ward','council_district','fire_zone','street'], axis=1, inplace=True)


In [12]:
pitt_police.drop(['inactive','inactive'], axis=1, inplace=True)

In [13]:
pitt_police

,name,neighborhood,police_zone,latitude,longitude
0,Municipal Courts Building,Bluff,2,40.434704,-79.994893
1,Police Headquarters,Chateau,1,40.450586,-80.021416
2,Police Zone 01,California-Kirkbride,1,40.456999,-80.016089
3,Police Zone 02,Crawford-Roberts,2,40.443576,-79.979969
4,Police Zone 03,Allentown,3,40.421662,-79.992493
5,Police Zone 04 Firehouse 18,Squirrel Hill North,4,40.441405,-79.921136
6,Police Zone 05,Highland Park,5,40.474220,-79.909146
7,Commercial Vehicle Inspections,Troy Hill,1,40.464773,-79.982321
8,Police Accident Investigation SWAT Building,Middle Hill,2,40.452735,-79.965726
9,Police Firing Range Building,Lincoln-Lemington-Belmar,5,40.479390,-79.906488


In [14]:
pitt_geo = r'/resources/data/pitt_neighborhoods.geojson'

pitt_map = folium.Map(location=[40.440624, -79.995888], zoom_start=12) #, tiles='Mapbox Bright')
pitt_map.choropleth(
    geo_data=pitt_geo,
    data=pitt_crime_rank,
    columns=['Neighborhood','Count'],
    key_on='feature.properties.hood',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Crime in Pittsburgh'
)

In [15]:
pitt_map

In [16]:
for lat,lon,area in zip(pitt_police['latitude'],pitt_police['longitude'],pitt_police['neighborhood']):
     folium.CircleMarker([lat, lon],
                            radius=1,
                            color='blue',
                            fill=True,
                            fill_opacity=0.7,
                            fill_color='blue',
                         label=["neighborhood"]
                           ).add_to(pitt_map)

In [17]:
pitt_map

In [35]:
from folium.features import DivIcon
colors = {'neighborhood' : 'blue'}
folium.Marker(location=[pitt_police["latitude"], pitt_police["longitude"]], icon=DivIcon(icon_size=(150,36), icon_anchor=(0,0),
        html='<div style="font-size: 16pt; color : {}">{}</div>'.format(colors pitt['neighborhood'],
                                                                        ['neighborhood']))).add_to(pitt_map)



NameError: name 'row' is not defined